In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = 127/30 * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = 3 * TR_C * TR_R
TR_R = tau / (3*TR_C)




# Logic Gates
g_inv, p_inv = 1, 323/127

g_nand2, g_nand3, g_nand6 = 4/3 , 5/3, 8/3
p_nand2, p_nand3, p_nand6 = p_inv*2, p_inv*3, p_inv*6

g_nor2, g_nor3 = 5/3, 7/3
p_nor2, p_nor3 = p_inv*2, p_inv*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [3]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

C_out = (WL_C + cell_C)
C_in = 5 * 1e-15 * 32
H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = ['value'], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,value
NAND6-INV,3.678072
NAND3-NOR2,3.707519


In [4]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,value
NAND6-INV Stage2,1.837400e-10
NAND3-NOR2 Stage2,1.644414e-10


In [5]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,value
NAND6-INV Stage4,1.574825e-10
NAND3-NOR2 Stage4,1.365706e-10


In [6]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND6-INV'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,value
NAND6-INV Stage6,1.778494e-10
NAND3-NOR2 Stage6,1.567217e-10


In [7]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,value
NAND6-INV Stage8,2.043526e-10
NAND3-NOR2 Stage8,1.828193e-10


In [8]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,value
NAND6-INV Stage2,1.837400e-10
NAND3-NOR2 Stage2,1.644414e-10
NAND6-INV Stage4,1.574825e-10
NAND3-NOR2 Stage4,1.365706e-10
NAND6-INV Stage6,1.778494e-10
NAND3-NOR2 Stage6,1.567217e-10
NAND6-INV Stage8,2.043526e-10
NAND3-NOR2 Stage8,1.828193e-10


In [9]:
# NMOS width of repeater drived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/(3*metal_r*TR_C) ) ** (1/2)

segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (3*TR_C*W) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (3*TR_C*p_inv*W) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 15 *1e-12 #measured FO1 delay
delay_WL = (N-1) * segment_delay + last_segment_delay   + (N-1) * repeater_delay
delay_WL

array([5.21895368e-09, 2.80395823e-09, 1.64761154e-09, 1.18563729e-09,
       1.18407583e-09, 1.62194366e-09, 2.67829071e-09, 4.81688653e-09])

In [10]:
delay_total = delay_decoder.copy()
for i in range(7):
  delay_total = pd.concat([delay_total, delay_decoder], axis=1)
delay_total = delay_total + delay_WL
delay_total.columns = [1,2,4,8,16,32,64,128]
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.402694e-09,2.987698e-09,1.831352e-09,1.369377e-09,1.367816e-09,1.805684e-09,2.862031e-09,5.000627e-09
NAND3-NOR2 Stage2,5.383395e-09,2.968400e-09,1.812053e-09,1.350079e-09,1.348517e-09,1.786385e-09,2.842732e-09,4.981328e-09
NAND6-INV Stage4,5.376436e-09,2.961441e-09,1.805094e-09,1.343120e-09,1.341558e-09,1.779426e-09,2.835773e-09,4.974369e-09
NAND3-NOR2 Stage4,5.355524e-09,2.940529e-09,1.784182e-09,1.322208e-09,1.320646e-09,1.758514e-09,2.814861e-09,4.953457e-09
NAND6-INV Stage6,5.396803e-09,2.981808e-09,1.825461e-09,1.363487e-09,1.361925e-09,1.799793e-09,2.856140e-09,4.994736e-09
NAND3-NOR2 Stage6,5.375675e-09,2.960680e-09,1.804333e-09,1.342359e-09,1.340797e-09,1.778665e-09,2.835012e-09,4.973608e-09
NAND6-INV Stage8,5.423306e-09,3.008311e-09,1.851964e-09,1.389990e-09,1.388428e-09,1.826296e-09,2.882643e-09,5.021239e-09
NAND3-NOR2 Stage8,5.401773e-09,2.986778e-09,1.830431e-09,1.368457e-09,1.366895e-09,1.804763e-09,2.861110e-09,4.999706e-09


In [11]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.389451e-09,2.974455e-09,1.818109e-09,1.356134e-09,1.354573e-09,1.792441e-09,2.848788e-09,4.987384e-09
std,2.095669e-11,2.095669e-11,2.095669e-11,2.095669e-11,2.095669e-11,2.095669e-11,2.095669e-11,2.095669e-11
min,5.355524e-09,2.940529e-09,1.784182e-09,1.322208e-09,1.320646e-09,1.758514e-09,2.814861e-09,4.953457e-09
25%,5.376246e-09,2.961251e-09,1.804904e-09,1.342930e-09,1.341368e-09,1.779236e-09,2.835583e-09,4.974179e-09
50%,5.390099e-09,2.975104e-09,1.818757e-09,1.356783e-09,1.355221e-09,1.793089e-09,2.849436e-09,4.988032e-09
75%,5.402003e-09,2.987008e-09,1.830661e-09,1.368687e-09,1.367125e-09,1.804993e-09,2.861340e-09,4.999936e-09
max,5.423306e-09,3.008311e-09,1.851964e-09,1.389990e-09,1.388428e-09,1.826296e-09,2.882643e-09,5.021239e-09


In [12]:
H

1.92

In [13]:
f_4

,value
NAND6-INV,3.577709
NAND3-NOR2,3.614408


In [14]:
W

array([0.64930219, 0.91825196, 1.29860437, 1.83650392, 2.59720874,
       3.67300783, 5.19441749, 7.34601566])

In [15]:
2*2.59720874

5.19441748